In [68]:
import pandas as pd
import numpy as np

In [69]:
e_fg_df = pd.read_csv("e_fg.txt", sep=",")


In [70]:
e_fg_df = e_fg_df[['Player', 'eFG%▼']]
e_fg_df.columns = ['Player', 'eFG%']

In [71]:
names = list(e_fg_df['Player'])
names = [item.partition("\\")[0] for item in names]

In [72]:
e_fg_df['Player'] = names

In [73]:
# link name and NBA player ID
text = open("Hackathon_player_names_matched_team.txt")
line = text.readline()
line = text.readline()
IDtoName = {} # ID as keys
NametoID = {} # name as keys

while line != "" :
    data = line.split()
    name = data[4] + " " + data[5]
    name = name.replace("\"","")
    ID = (float(data[3]))
    if ID not in IDtoName:
        IDtoName[ID] = name
    if name not in NametoID:
        NametoID[name] = ID
    line = text.readline()
    
#print(NametoID["LeBron James"])
#print(IDtoName[2544])


shot_df_name = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df_name = shot_df_name.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df_name = shot_df_name[shot_df_name['SEASON'] != "GAME_ID"]

shot_df_name = shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']]

#convert columns to numbers
shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']] = shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']].apply(pd.to_numeric).drop_duplicates()


names_df = pd.DataFrame(list(NametoID.items()), columns = ['NAME', 'PERSON_ID'])

names_df = pd.merge(names_df, shot_df_name,  how='left', left_on='PERSON_ID', right_on = 'PERSON_ID')

names_df = names_df.dropna()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
e_fg_df_all = names_df.merge(e_fg_df, how = "left", left_on = 'NAME', right_on = 'Player')

In [75]:
e_fg_df_all = e_fg_df_all[['NAME', 'eFG%', 'SV_PLAYER_ID']]

In [76]:
#0.502 is average shooting percentage. Might want to drop because we have the best shooters
e_fg_df_all.fillna(value = 0.502, inplace = True)

eFG% is done

In [77]:
blocks_df = pd.read_csv("blocks.txt", sep=",")
blocks_df = blocks_df[['Player', 'BLK▼']]
blocks_df.columns = ['Player', 'BLK']
names = list(blocks_df['Player'])
names = [item.partition("\\")[0] for item in names]
blocks_df['Player'] = names

In [78]:
master_df = e_fg_df_all.merge(blocks_df, how = "left", left_on = 'NAME', right_on = 'Player')

In [79]:
master_df.fillna(value = 0, inplace = True)
master_df = master_df[['NAME', 'eFG%', 'BLK', 'SV_PLAYER_ID']]

add advanced stats

In [80]:
adv_df = pd.read_csv("advanced_stats.txt", sep=",")
adv_df = adv_df[[
 'Player',
 'PER',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']]
names = list(adv_df['Player'])
names = [item.partition("\\")[0] for item in names]
adv_df['Player'] = names

In [82]:
master_df = master_df.merge(adv_df, how = "left", left_on = 'NAME', right_on = 'Player')

In [83]:
master_df.dropna()
master_df = master_df.drop(columns = 'Player')
master_df = master_df.drop_duplicates(subset='NAME', keep="first")

In [86]:
#some missing data for advanced stats. Rounded it to the average of the row
master_df.fillna(round(master_df.mean(), 3), inplace=True)

In [89]:
rpm_df = pd.read_csv("rpm.csv", sep=",")

In [94]:
rpm_df = rpm_df[['NAME', 'ORPM', 'DRPM', 'RPM']]
names = list(rpm_df['NAME'])
names = [item.partition(",")[0] for item in names]
rpm_df['NAME'] = names

In [95]:
master_df = master_df.merge(rpm_df, how = "left", left_on = 'NAME', right_on = 'NAME')

In [98]:
master_df.fillna(value = 0)

,NAME,eFG%,BLK,SV_PLAYER_ID,PER,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,ORPM,DRPM,RPM
0,Josh Smith,0.502,0.0,3834.0,10.600,-1.200,1.200,0.000,0.000,-3.600,1.600,-2.000,0.000,-3.14,1.05,-2.09
1,Marvin Williams,0.554,1.0,3928.0,16.800,4.700,3.200,7.800,0.161,1.700,1.000,2.700,2.700,2.08,0.59,2.67
2,Jason Terry,0.502,0.1,3333.0,10.200,1.100,0.400,1.500,0.057,-0.200,-2.600,-2.800,-0.300,-0.33,-2.81,-3.14
3,Kent Bazemore,0.517,0.5,5102.0,13.400,0.300,3.800,4.100,0.094,-1.300,1.600,0.300,1.200,-1.49,0.15,-1.34
4,Joe Johnson,0.509,0.0,3520.0,12.000,1.900,0.800,2.700,0.048,0.300,-1.600,-1.300,0.400,0.80,-0.24,0.56
5,Tim Hardaway,0.502,0.0,5194.0,11.700,0.900,1.000,1.900,0.105,-0.900,-1.200,-2.100,0.000,0.00,0.00,0.00
6,Thabo Sefolosha,0.502,0.5,4141.0,12.400,1.300,3.200,4.500,0.122,-1.700,2.800,1.100,1.400,-1.97,1.62,-0.35
7,Al Horford,0.547,1.5,4245.0,19.400,4.900,4.500,9.400,0.172,1.500,2.600,4.100,4.100,0.89,2.08,2.97
8,Lamar Patterson,0.502,0.0,5359.0,5.100,-0.300,0.500,0.200,0.025,-4.600,-0.300,-4.900,-0.300,-1.76,-0.53,-2.29
9,Alfred Horford,0.502,0.0,4245.0,13.457,1.398,1.292,2.687,0.083,-1.128,-0.335,-1.461,0.651,0.00,0.00,0.00
